<a href="https://colab.research.google.com/github/don05050505/don05050505/blob/main/LSTM230923.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Korpora
!pip install konlpy

In [ ]:
from konlpy.tag import Hannanum
from Korpora import Korpora
import pandas as pd
import numpy as np

In [ ]:
Korpora.fetch('nsmc')
corpus = Korpora.load('nsmc')

In [ ]:
train_text = corpus.train.texts
train_labels = corpus.train.labels
test_text = corpus.test.texts
test_labels = corpus.test.labels
Xtrain = pd.DataFrame(train_text, columns=['text'])
ytrain = pd.DataFrame(train_labels, columns=['labels'])
Xtest = pd.DataFrame(test_text, columns=['text'])
ytest = pd.DataFrame(test_labels, columns=['labels'])
print(Xtest)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
X = Xtest[:10000]
print(X)
y = ytest[:10000]
# 영문작업은 아래 바로 사용 가능
vector_layer = TextVectorization(
    max_tokens=200000,
    output_mode='int',
    output_sequence_length=20)
vector_layer.adapt(tf.cast(X, tf.string))
vocab_set = vector_layer.get_vocabulary()
vocab_size = len(vocab_set)
print(vocab_size)

In [ ]:
embedding_dim = 500
sequence_length = 20
lstm_length = 256
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(1,), dtype=tf.string))  # shape를 수정
model.add(vector_layer)
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=sequence_length))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_length)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 2진분류일 때
# model.add(tf.keras.layers.Dense(num_labels, activation='softmax'))  # 다진분류일 때
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # 2진분류일 때
# model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])  # 다진분류일 때


In [ ]:
model.summary()

In [ ]:
model.fit(X, y, epochs=10, validation_split=0.2, batch_size=32, verbose=1, shuffle=True)
